In [15]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 12:56:02 2022

@author: steph

Our simulation consists of 2 stocks 
we simulate their drift, volatility, and transition matrix
and plug them in max likelihood
before testing using MCMCs
"""

import numpy as np
import pandas as pd

##############################################################################
### VARIABLES ###
# drift = normal distribution(mean, sigma)
# dim(drift) = [stock] [state] 
# list of mean of stocks : [stock1, stock2]
mean_stocks = [0,0]
# list of sigma of stocks : [stock1, stock2]
sigma_stocks = []
sigma_states_elements = [0.5, 0.2, 0.5]

# randomly generated 2 stocks
num_stocks = 2

# we have 3 states: 
# state 0: crisis, state 1: stable, state 2: bubble
num_state = 3

# how many time (index of time)
N = 50

#fix the random seed
# np.random.seed(0)
#############################################################################
# b : drift for each stocks, for each state
# ex: b[0,1] is drift for stock 0 and state 1
b = np.random.randn(num_stocks, num_state)

# b0 < b1 < b2
for j in range(num_stocks):
    for i in range(num_state):
        if i == 0:
            b[j,i] = np.random.randn(1)
        else: 
            # make sure b of the next state is larger than the one before
            # b0 (crisis period) < b1 (stable) < b2 (bubble periode)
            b[j,i] = np.random.normal(b[j,i-1] +1, 0.5,1) 


print("drift matrix",b)

drift matrix [[-0.13702     1.35669224  1.9974174 ]
 [ 1.93803498  3.04823064  3.81068319]]


In [16]:
# volatility
sigma_stocks = [
    np.full((num_stocks, num_stocks), sigma_states_elements[x], dtype=np.float32)
    for x in range(num_state)
]
for x in sigma_stocks:
    np.fill_diagonal(x, 1)

sigma_stocks

[array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32),
 array([[1. , 0.2],
        [0.2, 1. ]], dtype=float32),
 array([[1. , 0.5],
        [0.5, 1. ]], dtype=float32)]

In [17]:
# create 3 matrix of dxd matrix
# for state 1 & 3 -> similar covariance matrix

# transition matrixs 
# P [stock][from state][to state]
P = [[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]], \
    [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]]

P

([[0.3, 0.4, 0.3], [0.1, 0.9, 0.0], [0.4, 0.3, 0.3]],
 [[0.2, 0.6, 0.2], [0.01, 0.9, 0.09], [0.4, 0.3, 0.3]])

In [18]:
# simulate y : state matrix
# t = 0 : first state : random between state 1,2,3
# y[i,t] : stock i, and t time
# ex: y[0,1] : state at stock 0 when time is in index 1.  

y = np.zeros((num_stocks, N))

for i in range(num_stocks):
    for t in range(N):
        if t == 0:
            y[i,t] = np.random.randint(0,3)
        else: 
            prev_state = int(y[i, t-1])
            y[i,t] = np.random.choice( np.arange(0,3), p=[P[i][prev_state][0], \
                                                          P[i][prev_state][1], \
                                                          P[i][prev_state][2]] )
y = y.astype('int') # state is always integer
print("state matrix", y)

state matrix [[0 2 1 1 1 1 1 1 1 1 1 0 1 1 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2
  1 1 1 1 1 1 1 1 0 2 0 0 0 2]
 [2 2 0 0 1 1 1 2 2 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [30]:
# DELTA of BROWNIAN MOTION is a normal distribution of mean 0 and std 1
# 1 stock have 1 brownian motion
# the length of 1 brownian motion is N
# brownian motion have dim(num_stock, N)

# brownian_motion_delta = np.zeros((num_stocks, N))

# for j in range(num_stocks): 
#     brownian_motion_delta[j] = np.random.standard_normal(N)

brownian_motion_delta = np.array([np.random.standard_normal(N) for _ in range(num_stocks)])
brownian_motion_delta

array([[ 0.0275097 ,  2.23201639, -0.1049797 ,  1.36741498, -1.65534404,
         0.15364446, -1.58447356,  0.84445431, -1.21286782,  0.28376955,
        -0.28219588, -1.15820319, -1.61935998, -0.51104046,  1.74062945,
        -0.29348505,  0.91722154, -0.05704287,  0.87672677, -1.82691138,
        -0.40318831,  0.94940552, -0.16325495, -0.08645528, -0.43046191,
         1.14937938,  0.29751435,  0.04402228,  0.64305455,  0.58822493,
         0.21258705,  1.5470315 , -0.06028753,  0.27808105, -0.64295255,
         0.15011523,  1.58776152, -0.6432576 , -1.13359283,  0.99675964,
        -0.14876615,  0.0960042 , -0.0451133 ,  0.07912172,  0.85053068,
        -0.83912419, -1.01177408,  0.08496814, -1.60643969, -1.37305354],
       [ 1.86668315,  0.75746833, -0.01005647,  1.23800694, -1.04059923,
        -0.31560312,  0.62345361,  0.89067168,  0.51291685, -2.54123881,
        -0.96808212,  0.47706809, -0.35595149,  2.54023162,  0.9265583 ,
         0.55808188, -1.11694955, -0.03529674,  0.

In [33]:
# r : rate of Return of stock price from timme 0 to time t
r = np.zeros((num_stocks, N))

for i in range(num_stocks):
    for n in range(1,N+1):
        sum_b = 0
        sum_sigma = 0
        for t in range(n):
            state_now = y[i,t]
            sum_b = sum_b + b[i , state_now]
            # Toby: what's the difference of d and i?
            for d in range(num_stocks):
                sum_sigma = sum_sigma + (sigma_stocks[state_now][i][d] * brownian_motion_delta[i, t])
            r[i,t] = sum_b + sum_sigma

print("return matrix", r)


return matrix [[-9.57554518e-02  5.24968653e+00  6.48040314e+00  9.47799336e+00
   8.84827275e+00  1.03893383e+01  9.84466231e+00  1.22146997e+01
   1.21159506e+01  1.38131663e+01  1.48312235e+01  1.29568987e+01
   1.23703590e+01  1.31138026e+01  1.55877268e+01  1.71449166e+01
   1.96022747e+01  2.08905155e+01  2.32992799e+01  2.24636785e+01
   2.33365448e+01  2.58325236e+01  2.69933099e+01  2.82462558e+01
   2.90863938e+01  3.18223413e+01  3.35360508e+01  3.49455697e+01
   3.70739274e+01  3.91364896e+01  4.07482863e+01  4.39614163e+01
   4.52457635e+01  4.69361530e+01  4.58347042e+01  4.80572944e+01
   5.13193005e+01  5.19040836e+01  5.19004645e+01  5.44532683e+01
   5.56314412e+01  5.71033385e+01  5.84058947e+01  5.98575330e+01
   6.09963091e+01  6.17350402e+01  6.00803590e+01  6.00707912e+01
   5.75241117e+01  5.74619488e+01]
 [ 6.61070791e+00  1.15575936e+01  1.34805439e+01  1.72755893e+01
   1.90751008e+01  2.17446077e+01  2.55409827e+01  3.06876734e+01
   3.52677318e+01  3.339390

In [14]:
# Price matrix S
# dimension: number_of_stock x N times
# ex: S[0,1] : stock 0 , time 1
S = np.zeros((num_stocks, N))
for i in range(num_stocks):
    for n in range(N):
        if n == 0:
            S[i,n] = np.random.randint(50,100)
        else:
            S[i,n] = S[i, n-1]*r[i,n]
print("price matrix", S)

price matrix [[8.70000000e+01 5.04555187e+02 2.93940137e+03 2.55937538e+04
  3.29124938e+05 4.89883235e+06 9.31164896e+07 1.99954848e+09
  4.80570951e+10 1.31448737e+12 3.94619187e+13 1.23927400e+15
  4.13511355e+16 1.46786279e+18 5.68763136e+19 2.44482754e+21
  1.11313671e+23 5.45636511e+24 2.79702783e+26 1.49115260e+28
  8.37641010e+29 4.83200818e+31 2.85296118e+33 1.70492290e+35
  1.07163283e+37 7.13392238e+38 5.02082143e+40 3.72855338e+42
  2.83876696e+44 2.26880344e+46 1.87132993e+48 1.60438188e+50
  1.38127807e+52 1.23355446e+54 1.12686278e+56 1.05547453e+58
  1.00004302e+60 9.75130608e+61 9.63563494e+63 9.77357221e+65
  1.00493974e+68 1.07713977e+70 1.18422637e+72 1.33490402e+74
  1.54283574e+76 1.77590876e+78 2.06642707e+80 2.46512607e+82
  3.00540609e+84 3.76787321e+86]
 [5.80000000e+01 2.34858240e+02 1.50900648e+03 1.31653347e+04
  1.59588995e+05 1.99237999e+06 2.91051582e+07 5.13023704e+08
  1.03482920e+10 2.26399035e+11 5.46246295e+12 1.58313722e+14
  4.96481311e+15 1.64716